# Get game expansions to eliminate overrepresentation

In [3]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
import requests
import xml.etree.ElementTree as ET
import time
import csv

In [4]:
def con_cur_to_bgg_db():
    con = pg2.connect(host='34.216.22.202',
                  dbname='postgres',
                  user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur
    
def insert_re_ex(exp_list):
    con, cur = con_cur_to_bgg_db()
    cur.execute('INSERT INTO re_ex VALUES\
            {};'.format(", ".join(repr(e) for e in exp_list)))
    con.commit()
    con.close()

def get_id_list(page_num):
    ids = []

    with open('../games/page_{}'.format(page_num, 'r')) as f:
        reader = csv.reader(f)
        for row in reader:
            ids.append(row[0])
            
    api_id_list = ','.join(ids)  
    return api_id_list, ids

def get_game_tree(api_id_list):
    api_q = 'http://www.boardgamegeek.com/xmlapi/boardgame/{}?stats=1'.format(api_id_list)
    r = requests.get(api_q)
    tree = ET.fromstring(r.text)

    return tree

def get_game_expansions(gid, tree):

    root = tree.find("boardgame[@objectid='{}']".format(gid))

    reimp = [item.attrib['objectid'] for item in root.findall('boardgameimplementation')]

    exp = [item.attrib['objectid'] for item in root.findall('boardgameexpansion')]
    
    exp_list = []
    exp_list.extend(list(zip([gid]*len(exp), exp)))
    exp_list.extend(list(zip([gid]*len(reimp), reimp)))
    
    if exp_list:
        insert_re_ex(exp_list)

In [ ]:
for page in range(1, 101):
    api_id_list, ids = get_id_list(page)
    tree = get_game_tree(api_id_list)
    for gid in ids:
        get_game_expansions(gid, tree)
    time.sleep(1)